In [1]:
__author__='Florin Bora'

import player
import game
import neural_network
import mcts
import board as b
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt


In [21]:
# mcts.MCTS.get_tree_and_edges(reset=True)
# neural_network.nn_predictor.reset_nn_check_pts()
# nn_training_set = None

iterations = 50
player1 = player.Zero_Player('x', 'Bot_ONE', nn_type='best', temperature=1)
player2 = player.Zero_Player('o', 'Bot_ONE', nn_type='best', temperature=1)

player1.load_keras()
player2.load_keras()


# player1.value_estimate = ""
self_play_game = game.Game(player1, player2)
self_play_results = self_play_game.play(2)

augmented_self_play_results = neural_network.augment_data_set(self_play_results)

mcts.MCTS.update_mcts_edges(augmented_self_play_results)
tree, edge_statistics = mcts.MCTS.get_tree_and_edges()

In [22]:
master_df = neural_network.create_data_from_mcts(edge_statistics)

wm = lambda x: np.average(x, weights=master_df.loc[x.index, "N"])
weightd_values = master_df.groupby(["final_state"])["value"].apply(wm).reset_index()
weightd_values.columns = ["final_state", "value_weighted"]
master_df = master_df.merge(weightd_values, on="final_state")
master_df.shape

(3511, 7)

In [ ]:
augmented_self_play_results[2] 

In [ ]:
self_play_results[0][0]

In [9]:
board = b.Board()
str_arr = board.arr2str(self_play_results[0][0][2])

In [25]:
model = keras.models.load_model('./best_keras_model.tf')

In [53]:
test = np.array([[  1,   0 ,  1],
                 [  0,  0 , 0],
                 [  -1,   0,  -1]] )

n_map = {-1:"o", 0: " ", 1: "x"}

str_state = ''.join([n_map[i] for i in test.reshape(9,)])
str_state

pred, prob = model.predict(np.stack([[test]],-1))
print(pred.round(2))
prob.round(2)

[[0.34 0.55 0.11]]


array([[0.  , 0.15, 0.  , 0.04, 0.25, 0.01, 0.  , 0.55, 0.  , 0.  ]],
      dtype=float32)

In [50]:
master_df[master_df["final_state"]==str_state]

,final_state,value,N,init_state,move,inv_freq,value_weighted


In [23]:
master_df

,final_state,value,N,init_state,move,inv_freq,value_weighted
0,xox xo,1.0,1,xox xo,6.0,877.75,0.0
1,xox xo,1.0,1,xox xo,5.0,877.75,0.0
2,xox xo,-1.0,1,xox xo,6.0,877.75,0.0
3,xox xo,-1.0,1,xox xo,5.0,877.75,0.0
4,x x oo,1.0,1,x x oo,1.0,3511.00,1.0
...,...,...,...,...,...,...,...
3506,x o o x x,1.0,1,x o o x x,5.0,3511.00,1.0
3507,oxxxoxo,-1.0,1,oxxxoxo,8.0,3511.00,-1.0
3508,o ooxx x,1.0,1,o ooxx x,2.0,3511.00,1.0
3509,ox xoox,-1.0,1,ox xoox,0.0,3511.00,-1.0
